In [1]:
!pip install flask-ngrok

# Train MLP

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

## preprocess data

In [3]:
# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)

In [4]:
X

,Foundation_rank,HeatingQC_rank,ExterQual_rank,CentralAir_rank,PavedDrive_rank,HouseStyle_rank,Exterior2nd_rank,Neighborhood_rank,KitchenAbvGr,EnclosedPorch,ExterCond_rank,Exterior1st_rank,MSZoning_rank,KitchenQual_rank,BldgType_rank,Functional_rank,Heating_rank,MSSubClass,OverallCond,Condition1_rank,Street_rank,YrSold,LowQualFinSF,MiscVal,BsmtHalfBath,Utilities_rank,BsmtFinSF2,Condition2_rank,RoofMatl_rank,3SsnPorch,MoSold,LandSlope_rank,SaleType_rank,LandContour_rank,PoolArea,LotConfig_rank,ScreenPorch,SaleCondition_rank,RoofStyle_rank,BedroomAbvGr,BsmtUnfSF,BsmtFullBath,LotArea,LotShape_rank,HalfBath,OpenPorchSF,2ndFlrSF,WoodDeckSF,BsmtFinSF1,Fireplaces,YearRemodAdd,YearBuilt,TotRmsAbvGrd,FullBath,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual
0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,60,5,1,1,2008,0,0,0,1,0,1,1,0,2,1,1,1,0,1,0,1,1,3,150,1,8450,1,1,61,854,0,706,0,2003,2003,8,2,856,856,548,2,1710,7
1,2,1,2,1,1,2,2,2,1,0,1,2,1,2,1,1,1,20,8,2,1,2007,0,0,1,1,0,1,1,0,5,1,1,1,0,2,0,1,1,3,284,0,9600,1,0,0,0,298,978,1,1976,1976,6,2,1262,1262,460,2,1262,6
2,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,60,5,1,1,2008,0,0,0,1,0,1,1,0,9,1,1,1,0,1,0,1,1,3,434,1,11250,2,1,42,866,0,486,1,2002,2001,6,2,920,920,608,2,1786,7
3,3,2,2,1,1,1,3,3,1,272,1,3,1,1,1,1,1,70,5,1,1,2006,0,0,0,1,0,1,1,0,2,1,1,1,0,3,0,2,1,3,540,1,9550,2,0,35,756,0,216,1,1970,1915,7,1,961,756,642,3,1717,7
4,1,1,1,1,1,1,1,4,1,0,1,1,1,1,1,1,1,60,5,1,1,2008,0,0,0,1,0,1,1,0,12,1,1,1,0,2,0,1,1,4,490,1,14260,2,1,84,1053,192,655,1,2000,2000,9,2,1145,1145,836,3,2198,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,1,2,1,1,1,1,18,1,0,1,1,1,2,1,1,1,60,5,1,1,2007,0,0,0,1,0,1,1,0,8,1,1,1,0,1,0,1,1,3,953,0,7917,1,1,40,694,0,0,1,2000,1999,7,2,953,953,460,2,1647,6
1456,2,3,2,1,1,2,5,7,1,0,1,8,1,2,1,2,1,20,6,1,1,2010,0,0,0,1,163,1,1,0,2,1,1,1,0,1,0,1,1,3,589,1,13175,1,0,0,0,349,790,2,1988,1978,7,2,2073,1542,500,2,2073,6
1457,6,1,3,1,1,1,7,3,1,0,2,7,1,1,1,1,1,70,9,1,1,2010,0,2500,0,1,0,1,1,0,5,1,1,1,0,1,0,1,1,4,877,0,9042,1,0,60,1152,0,275,2,2006,1941,9,2,1188,1152,252,1,2340,7
1458,2,2,2,1,1,2,2,12,1,112,1,2,1,1,1,1,1,20,6,1,1,2010,0,0,0,1,1029,1,1,0,4,1,1,1,0,1,0,1,2,2,0,1,9717,1,0,0,0,366,49,0,1996,1950,5,1,1078,1078,240,1,1078,5


In [5]:
# select features

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg

X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

In [6]:
X.shape

(1460, 8)

In [7]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [8]:
scaled_X.shape, scaled_y.shape

((1460, 8), (1460, 1))

## Training

In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=scaled_X.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)
model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y,
          batch_size=2, epochs=150,
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0050 - val_loss: 0.0018
Epoch 2/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0032 - val_loss: 0.0019
Epoch 3/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 4/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0029 - val_loss: 0.0016
Epoch 5/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0019
Epoch 6/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0028 - val_loss: 0.0015
Epoch 7/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0026 - val_loss: 0.0015
Epoch 8/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0024 - val_loss: 0.0013
Epoch 9/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0026 - val_loss: 0.0020
Epoch 10/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0024 - val_lo

In [12]:
 scaled_X.shape

(1460, 8)

In [13]:
pred = model.predict(scaled_X[:5])
pred = y_min_max_scaler.inverse_transform(pred)

In [16]:
pred

array([[177153.6 ],
       [149654.47],
       [201769.31],
       [196613.1 ],
       [278938.75]], dtype=float32)

In [15]:
str(pred[0][0])

'177153.6'

# Flask server

In [17]:
# Read data (v)
# Preprocess data
# Model prediction
# Return predict

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  # TODO: 전처리
  return np.zeros((1, 8)) # dummy data

@app.route("/")
def predict():
  # return "<h1>This is your Flask server.</h1>"
  return render_template("submit_form.html")

@app.route("/result", methods=['POST'])
def result():
  data = request.form

  message = ""
  message += "<h1>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"
  
  # 데이터 전처리
  X = preprocess_data(data) # data: user가 보낸 것
  # X: (1, 8) zeros

  pred = model.predict(X)
  pred = y_min_max_scaler.inverse_transform(pred)
  # array (1, 1) --> string

  message += "</br>"
  message += "Predicted price: " + str(pred[0][0])

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ce769e656715.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Aug/2021 03:37:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 03:37:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Aug/2021 03:37:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 03:37:16] "POST /result HTTP/1.1" 200 -


OverallQual 6
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1


127.0.0.1 - - [11/Aug/2021 03:37:18] "GET /result HTTP/1.1" 405 -
127.0.0.1 - - [11/Aug/2021 03:37:23] "POST /result HTTP/1.1" 200 -


OverallQual 1
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
